In [70]:
%load_ext sql
%sql sqlite:///HW1.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @HW1.db'

Problem - U.S. Sustainable Energy Sources
----------------------------------------------

We've prepared and loaded a [public dataset](https://catalog.data.gov/dataset/energy-generation-by-state-and-technology-2009-69f4f) from the US DOE (Department of Energy) of sustainable energy production in MWh (megawatt hours, i.e., 1000 [kilowatt hours](https://en.wikipedia.org/wiki/Kilowatt_hour)). This data is from 2009, the latest year available.  The data includes each state, the region of the United States it is in, and its production in MWh of sustainable energy by source (solar, wind, hydroelectric, and nuclear). The table `energy` has the following schema:
```
TABLE energy (
    state varchar(30),
    region varchar(30),
    solar float,
    wind float,
    hydro float,
    nuclear float)
```

In [71]:
%sql SELECT * FROM energy LIMIT 3;

 * sqlite:///HW1.db
Done.


state,region,solar,wind,hydro,nuclear
Alabama,Southeast,0.0,0.0,11753493.37,39716204.0
Alaska,Alaska,0.0,3062.442,1204550.392,0.0
Arizona,West,13759.445,9555.0,6348462.992,30661851.0


### Part (a): Regional Champions

Using a _single SQL query_, find all of the regions in the United States with a state in it that is the leading producer of one of the four types of energy (solar, wind, hydro, and nuclear), and return the counts of how many state winners they had in descending order. **Do not include any regions with no state winners.**

Further requirements:
* Use `GROUP BY`
* Write the shortest possible SQL query to accomplish this
* Return relation `(region, num_state_winners)`

Write your query here:

In [72]:
%%sql
SELECT region, count(region) AS num_state_winners 
FROM (SELECT state, region, MAX(solar)
FROM energy
UNION ALL
SELECT state, region, MAX(wind)
FROM energy
UNION ALL
SELECT state, region, MAX(hydro)
FROM energy
UNION ALL
SELECT state, region, MAX(nuclear)
FROM energy)
GROUP BY region
ORDER BY num_state_winners DESC

 * sqlite:///HW1.db
Done.


region,num_state_winners
West,2
Midwest,1
Southeast,1


In [73]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

"\nExpected output below- don't re-evaluate this cell!\n\nNOTE: A valid answer must work for ALL inputs of the given type,\nnot just this example.  I.e. do not hardcode around this answer / etc!\n"

### Part (b): Pareto Frontiers

Solar power and wind power [tend to be complementary](https://en.wikipedia.org/wiki/Wind_power#Variability), since it tends to be less windy when there are few clouds and the sun can best reach solar panels.

Our goal in this part is to identify states that strike the best balance between solar and wind energy production. Here we define a state as "best" if it exists on the [Pareto frontier](https://en.wikipedia.org/wiki/Pareto_efficiency#Formal_representation) of solar and wind energy production. In other words, a state is Pareto optimal if no other state produces **both more solar and more wind energy**, and the Pareto frontier is the set of states that are Pareto optimal.

Write a query that returns the entire Pareto frontier. Results should be triples of the form `(state, solar, wind)`, where `state` is the name of the state in the frontier, and `solar` and `wind` are its solar and wind energy production in MWh. Order the results in descending order by sum total of solar and wind energy production in MWh.

Write your query here:

In [74]:
%%sql
SELECT state, solar, wind
FROM energy
WHERE 
wind>= (select wind from energy where solar >= (select max(solar) as solar from energy)) 
and solar>= (select solar from energy where wind >= (select max(wind) as solar from energy)) 
ORDER BY solar+wind desc

 * sqlite:///HW1.db
Done.


state,solar,wind
Texas,0.0,19367238.86
Iowa,0.0,7331390.872
California,611763.387,5764637.309


In [75]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

"\nExpected output below- don't re-evaluate this cell!\n\nNOTE: A valid answer must work for ALL inputs of the given type,\nnot just this example.  I.e. do not hardcode around this answer / etc!\n"

### Part (c)

Find a list of regions which had a minimum state nuclear power greater than _10% of the maximum state nuclear power value_ (where _minimum state nuclear power_ = minimum value of nuclear power production over all states with non-zero nuclear production).

*Note: do not hard-code the maximum state nuclear power or any other input-data-dependent values.*

**Do this using `GROUP BY` and aggregate functions (e.g. `COUNT`, `SUM`, `MAX`, `MIN`)**.  Write your query here:

In [76]:
%%sql
SELECT region
from energy
where nuclear > 0
group by region
having
min(nuclear)>max(nuclear) *0.1

 * sqlite:///HW1.db
Done.


region
Heartland
Mid Atlantic
New England
Southeast
West


In [77]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

"\nExpected output below- don't re-evaluate this cell!\n\nNOTE: A valid answer must work for ALL inputs of the given type,\nnot just this example.  I.e. do not hardcode around this answer / etc!\n"

### Part (d)

Do the same as above, except do **not** use `GROUP BY` or any aggregate functions.  Write your query here:

In [78]:
import sqlite3
import pandas as pd

con = sqlite3.connect('HW1.db')
SQL_Query = pd.read_sql_query('select region, nuclear from energy', con)
df = pd.DataFrame(SQL_Query)
df = df[df.nuclear>0]
output = []

for i in list(df.region.unique()):
    if (df[df.region==i].sort_values(by = ['nuclear']).iloc[0, 1] >= 
     df[df.region==i].sort_values(by = ['nuclear'], ascending = False).iloc[0, 1] * 0.1):
        output.append(i)

print(output)

['Southeast', 'West', 'New England', 'Heartland', 'Mid Atlantic']


In [79]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

"\nExpected output below- don't re-evaluate this cell!\n\nNOTE: A valid answer must work for ALL inputs of the given type,\nnot just this example.  I.e. do not hardcode around this answer / etc!\n"